# データのインプット

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# GBDT

In [4]:
# データの取得
train2 = train.copy()
test2 = test.copy()

# 目的変数と特徴量に分ける
train2_x = train2.drop(['Survived'], axis=1)
train2_y = train['Survived']

# テストデータは特徴量のみ
test2_x = test2.copy()

In [5]:
train2.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 特徴量の作成

In [6]:
from sklearn.preprocessing import LabelEncoder

# 変数PassegerIdを取得する
train2_x = train2_x.drop(['PassengerId'], axis=1)
test2_x = test2_x.drop(['PassengerId'], axis=1)

In [7]:
# 変数Name, Ticket, Cabinを除外する
train2_x = train2_x.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test2_x = test2_x.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [8]:
# それぞれのカテゴリ変数にlabel encordingを適用する
for c in ['Sex', 'Embarked']:
    le = LabelEncoder()
    le.fit(train2_x[c].fillna('NA'))
    
    # 学習データ、テストデータを変換する
    train2_x[c] = le.transform(train2_x[c].fillna('NA'))
    test2_x[c] = le.transform(test2_x[c].fillna('NA'))

## モデルの作成
xgboostを使用

In [9]:
from xgboost import XGBClassifier

# モデルの作成および学習データを与えて学習
model = XGBClassifier(n_estimators=20, random_state=71)
model.fit(train2_x, train2_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=None,
              num_parallel_tree=None, random_state=71, ...)

In [10]:
# テストデータの予測値を確率で出力する
pred = model.predict_proba(test2_x)[:,1]

In [11]:
# テストデータの予測値を二値に変換する
pred_label = np.where(pred>0.5,1,0)

In [12]:
# 提出用のファイルの作成
submission = pd.DataFrame({'Survived':pred_label})
submission.insert(0, 'PassengerId', test['PassengerId'])
submission.to_csv('titanic_answer_02.csv', index=False)

In [13]:
# 回答ファイルの作成・エクスポート
# PassengerId を取得
PassengerId2 = np.array(test2["PassengerId"]).astype(int)

# pred_label と PassengerId を結合
answer2 = pd.DataFrame(pred_label, PassengerId2, columns = ["Survived"])

# titanic_answer_01.csv としてエクスポート
answer2.to_csv("titanic_answer_02.csv", index_label = ["PassengerId"])

# 提出用ファイルの確認

In [14]:
gender_submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [15]:
answer2.head()

,Survived
892,0
893,0
894,0
895,0
896,1
